## Memory
Helping LLMs remember information.

Memory is a bit of a loose term. It could be as simple as remembering information you've chatted about in the past or more complicated information retrieval.

We'll keep it towards the Chat Message use case. This would be used for chat bots.

There are many types of memory, explore [the documentation](https://python.langchain.com/en/latest/modules/memory/how_to_guides.html) to see which one fits your use case.

In [6]:
import os
from dotenv import load_dotenv, find_dotenv
import langchain
langchain.debug = True
load_dotenv(find_dotenv())

openai_api_key = os.environ["OPENAI_API_KEY"]

### Chat Message History

In [7]:
from langchain.memory import ChatMessageHistory
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)

history = ChatMessageHistory()

history.add_ai_message("hi!")

history.add_user_message("what is the capital of france?")

In [8]:
history.messages

[AIMessage(content='hi!'),
 HumanMessage(content='what is the capital of france?')]

In [9]:
ai_response = chat.invoke(history.messages)
ai_response

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "AI: hi!\nHuman: what is the capital of france?"
  ]
}
[llm/end] [llm:ChatOpenAI] [1.05s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The capital of France is Paris.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "The capital of France is Paris.",
            "response_metadata": {
              "token_usage": {
                "completion_tokens": 7,
                "prompt_tokens": 20,
                "total_tokens": 27
              },
              "model_name": "gpt-3.5-turbo-0125",
              "system_fingerprint": null,
              

AIMessage(content='The capital of France is Paris.', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 20, 'total_tokens': 27}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-69b29c82-6e6c-4526-8263-446d6d1332d0-0', usage_metadata={'input_tokens': 20, 'output_tokens': 7, 'total_tokens': 27})

In [5]:
ai_response.content

'The capital of France is Paris.'

In [10]:
history.add_ai_message(ai_response.content)
history.messages

[AIMessage(content='hi!'),
 HumanMessage(content='what is the capital of france?'),
 AIMessage(content='The capital of France is Paris.')]

In [12]:
from langchain.callbacks import get_openai_callback
def count_tokens(chain, query):
    # Get the OpenAI callback handler in a context manager.
    # which conveniently exposes token and cost information.
    with get_openai_callback() as cb:
        result = chain.invoke(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

### Conversation Buffer Memory
The conversation buffer memory automatically remembers the last messages in the conversation.

In [13]:
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferMemory

llm = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)

conversation_buf = ConversationChain(llm=llm,verbose=True, memory=ConversationBufferMemory())

count_tokens(conversation_buf,"Hi, i am not able to start my macbook laptop")


[chain/start] [chain:ConversationChain] Entering Chain run with input:
{
  "input": "Hi, i am not able to start my macbook laptop",
  "history": ""
}
[llm/start] [chain:ConversationChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n\nHuman: Hi, i am not able to start my macbook laptop\nAI:"
  ]
}
[llm/end] [chain:ConversationChain > llm:ChatOpenAI] [1.61s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Hello! I'm sorry to hear that you're having trouble with your MacBook. Can you provide me with more details about what happens when you try to start it up? Are you seeing any error messages or is the screen completely blank?",
        "generation_info": {
          "finis

{'input': 'Hi, i am not able to start my macbook laptop',
 'history': '',
 'response': "Hello! I'm sorry to hear that you're having trouble with your MacBook. Can you provide me with more details about what happens when you try to start it up? Are you seeing any error messages or is the screen completely blank?"}

In [14]:
count_tokens(conversation_buf,"It is a macbook pro 2021 model?")

[chain/start] [chain:ConversationChain] Entering Chain run with input:
{
  "input": "It is a macbook pro 2021 model?",
  "history": "Human: Hi, i am not able to start my macbook laptop\nAI: Hello! I'm sorry to hear that you're having trouble with your MacBook. Can you provide me with more details about what happens when you try to start it up? Are you seeing any error messages or is the screen completely blank?"
}
[llm/start] [chain:ConversationChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\nHuman: Hi, i am not able to start my macbook laptop\nAI: Hello! I'm sorry to hear that you're having trouble with your MacBook. Can you provide me with more details about what happens when you try to start it up?

{'input': 'It is a macbook pro 2021 model?',
 'history': "Human: Hi, i am not able to start my macbook laptop\nAI: Hello! I'm sorry to hear that you're having trouble with your MacBook. Can you provide me with more details about what happens when you try to start it up? Are you seeing any error messages or is the screen completely blank?",
 'response': "That's great to know! The MacBook Pro 2021 model is a powerful and sleek laptop with impressive performance capabilities. When you try to start it up, do you hear any sounds like the fan spinning or the startup chime? Have you tried connecting it to a power source to ensure it's not a battery issue? Let's troubleshoot this together."}

In [15]:
count_tokens(conversation_buf,"How do i start it up")

[chain/start] [chain:ConversationChain] Entering Chain run with input:
{
  "input": "How do i start it up",
  "history": "Human: Hi, i am not able to start my macbook laptop\nAI: Hello! I'm sorry to hear that you're having trouble with your MacBook. Can you provide me with more details about what happens when you try to start it up? Are you seeing any error messages or is the screen completely blank?\nHuman: It is a macbook pro 2021 model?\nAI: That's great to know! The MacBook Pro 2021 model is a powerful and sleek laptop with impressive performance capabilities. When you try to start it up, do you hear any sounds like the fan spinning or the startup chime? Have you tried connecting it to a power source to ensure it's not a battery issue? Let's troubleshoot this together."
}
[llm/start] [chain:ConversationChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: The following is a friendly conversation between a human and an AI. The AI is talkative and provides

{'input': 'How do i start it up',
 'history': "Human: Hi, i am not able to start my macbook laptop\nAI: Hello! I'm sorry to hear that you're having trouble with your MacBook. Can you provide me with more details about what happens when you try to start it up? Are you seeing any error messages or is the screen completely blank?\nHuman: It is a macbook pro 2021 model?\nAI: That's great to know! The MacBook Pro 2021 model is a powerful and sleek laptop with impressive performance capabilities. When you try to start it up, do you hear any sounds like the fan spinning or the startup chime? Have you tried connecting it to a power source to ensure it's not a battery issue? Let's troubleshoot this together.",
 'response': "To start up your MacBook Pro 2021 model, simply press the power button located on the keyboard. It's usually located at the top right corner of the keyboard. If the laptop doesn't turn on, try connecting it to a power source using the charger that came with the laptop. Make 

In [16]:
conversation_buf.memory.buffer

"Human: Hi, i am not able to start my macbook laptop\nAI: Hello! I'm sorry to hear that you're having trouble with your MacBook. Can you provide me with more details about what happens when you try to start it up? Are you seeing any error messages or is the screen completely blank?\nHuman: It is a macbook pro 2021 model?\nAI: That's great to know! The MacBook Pro 2021 model is a powerful and sleek laptop with impressive performance capabilities. When you try to start it up, do you hear any sounds like the fan spinning or the startup chime? Have you tried connecting it to a power source to ensure it's not a battery issue? Let's troubleshoot this together.\nHuman: How do i start it up\nAI: To start up your MacBook Pro 2021 model, simply press the power button located on the keyboard. It's usually located at the top right corner of the keyboard. If the laptop doesn't turn on, try connecting it to a power source using the charger that came with the laptop. Make sure the charger is plugged 

### Conversation Summary Memory

In [21]:
from langchain.chains.conversation.memory import ConversationSummaryMemory
memory = ConversationSummaryMemory(llm=llm)
conversation_sum = ConversationChain(llm=llm, memory=memory,verbose=True)
langchain.debug = False

In [22]:
count_tokens(conversation_sum,"Hi, i am not able to start my macbook laptop")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, i am not able to start my macbook laptop
AI:

> Finished chain.
Spent a total of 318 tokens


{'input': 'Hi, i am not able to start my macbook laptop',
 'history': '',
 'response': "Hello! I'm sorry to hear that you're having trouble with your MacBook. Can you provide me with more details about what happens when you try to start it up?"}

In [24]:
count_tokens(conversation_sum,"How do i start it up")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human is having trouble starting their MacBook laptop. The AI expresses sympathy and asks for more details about the issue, such as what happens when they try to turn it on and if they see any lights or hear any sounds.
Human: How do i start it up
AI:

> Finished chain.
Spent a total of 487 tokens


{'input': 'How do i start it up',
 'history': 'The human is having trouble starting their MacBook laptop. The AI expresses sympathy and asks for more details about the issue, such as what happens when they try to turn it on and if they see any lights or hear any sounds.',
 'response': "I'm sorry to hear that you're having trouble starting up your MacBook. Can you tell me what happens when you try to turn it on? Do you see any lights on the laptop or hear any sounds? This information will help me better understand the issue and provide you with the right solution."}

# Workshop exercise:
Create a chat with a memory that uses ConversationSummaryMemory that chat should be able to answer questions about dogs brands and their characteristics.

e.g. "What kind of dog is a labrador?"